<a href="https://colab.research.google.com/github/NoraHK3/DataSciProject/blob/main/Data_cleaning_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **1- CSV Translation**

The original dataset contained a mix of **Arabic and English** text — some dish names and ingredients were already in English, while others were still in Arabic.
To make the data consistent and ready for analysis, this step translates **all Arabic parts** into **English**, while keeping the existing English text as it is.

It uses **Google Translate** through the `deep-translator` library, with extra tools to keep translations accurate:

* **Overrides (`overrides_expanded.json`)** – custom manual translations that replace Google’s output for specific words (e.g., “ليمون أسود” → “black lime”).
* **Cache (`translation_cache.csv`)** – stores previous translations to keep results consistent and faster.
* **Post-fixes** – fixes common translation mistakes automatically (e.g., “black lemon” → “black lime”).
* **Classification handling** – splits multi-part text like “رز | دجاج” and translates each piece separately (“rice | chicken”).

**Why we did this:**
Mixing two languages made the data inconsistent. Translating everything to English first ensures that the later cleaning and ingredient normalization steps work correctly and uniformly.

**Output:**
 `SaudiFoodFile_english_FIXED.csv` — fully English, consistent version of the dataset


In [1]:
OVERRIDES_JSON = "overrides_expanded.json"

In [2]:
# ============================================
# Translator (Deep) with Expanded Overrides + Cache Purge + Debug
# ============================================

!pip install -q pandas deep-translator

import os, re, json, pandas as pd
from deep_translator import GoogleTranslator

INPUT_CSV  = "SaudiFoodFile.csv"
OUTPUT_CSV = "SaudiFoodFile_english_FIXED.csv"
CACHE_CSV  = "translation_cache.csv"

# Prefer expanded overrides if present
OVR_EXP   = "overrides_expanded.json"
OVR_BASE  = "overrides.json"
OVERRIDES_JSON = OVR_EXP if os.path.exists(OVR_EXP) else OVR_BASE

HANDLE_CLASSIFICATIONS = True  # split 'classifications' by '|'
TRANSLATE_COLS = None          # None -> all object columns

# ---------- helpers ----------
AR_DIAC = re.compile(r"[\u0610-\u061A\u064B-\u065F\u06D6-\u06ED]")
def norm_ar(s: str) -> str:
    s = AR_DIAC.sub("", s)
    s = s.replace("\u0640","")
    s = s.replace("أ","ا").replace("إ","ا").replace("آ","ا")
    s = s.replace("ى","ي").replace("ئ","ي").replace("ؤ","و").replace("ٱ","ا")
    return s

def key_norm(x: str) -> str:
    return norm_ar(str(x).strip().lower())

POST_FIX = {
    "black lemon": "black lime",
    "nail": "cloves",
    "cardamon": "cardamom",
    "yougurt": "yogurt",
    "youghurt": "yogurt",
}

def apply_postfix(en: str) -> str:
    return POST_FIX.get(str(en).strip().lower(), str(en).strip())

# ---------- load data ----------
# CSV
try:
    df = pd.read_csv(INPUT_CSV, encoding="utf-8")
except UnicodeDecodeError:
    df = pd.read_csv(INPUT_CSV, encoding="cp1256")

# Overrides
if os.path.exists(OVERRIDES_JSON):
    with open(OVERRIDES_JSON, "r", encoding="utf-8") as f:
        OV = json.load(f)
else:
    OV = {}

# Normalized override view (for Arabic variant matching)
OV_NORM = {key_norm(k): v for k, v in OV.items() if re.search(r"[\u0600-\u06FF]", k)}
print(f"🔧 Using overrides file: {OVERRIDES_JSON}")
print(f"   Loaded overrides: {len(OV)} (normalized Arabic keys: {len(OV_NORM)})")
# show a few samples for sanity
for i,(k,v) in enumerate(list(OV.items())[:8]):
    print(f"   • {k}  ->  {v}")
    if i>=7: break

# Cache (load then purge entries that now have overrides)
if os.path.exists(CACHE_CSV):
    cache_df = pd.read_csv(CACHE_CSV)
    CACHE = dict(cache_df.values)  # {raw: english}
else:
    CACHE = {}

def override_lookup(text: str):
    if text in OV:
        return OV[text]
    kn = key_norm(text)
    if kn in OV_NORM:
        return OV_NORM[kn]
    return None

# Purge cache entries that should now be overridden
purged = 0
to_del = []
for raw in list(CACHE.keys()):
    if override_lookup(raw):
        to_del.append(raw)
for raw in to_del:
    CACHE.pop(raw, None)
    purged += 1
print(f"🧹 Purged {purged} cache entries that now have overrides")

translator = GoogleTranslator(source="auto", target="en")

def translate_text(text: str) -> str:
    if pd.isna(text) or str(text).strip() == "":
        return text
    s = str(text).strip()

    # 1) override wins (exact or normalized)
    ov = override_lookup(s)
    if ov:
        return ov

    # 2) cache
    if s in CACHE:
        return CACHE[s]

    # 3) machine translation
    try:
        en = translator.translate(s) or s
        en = apply_postfix(en)
    except Exception:
        en = s  # keep original on error

    CACHE[s] = en
    return en

def translate_classifications_cell(cell: str) -> str:
    parts = [p.strip() for p in str(cell).split("|")]
    out = []
    for p in parts:
        if not p:
            continue
        ov = override_lookup(p)
        en = ov if ov else translate_text(p)
        out.append(str(en).lower())
    return " | ".join(out)

# ---------- choose columns ----------
obj_cols = [c for c in df.columns if df[c].dtype == "object"]
cols = obj_cols if TRANSLATE_COLS is None else [c for c in TRANSLATE_COLS if c in df.columns]
print(f"📝 Translating columns: {cols}")

# ---------- translate ----------
for col in cols:
    print(f"➡️  Translating: {col}")
    if HANDLE_CLASSIFICATIONS and col.lower() == "classifications":
        df[col] = df[col].astype(str).apply(translate_classifications_cell)
    else:
        df[col] = df[col].apply(translate_text)

# ---------- save ----------
df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")
pd.DataFrame(list(CACHE.items()), columns=["raw","english"]).to_csv(CACHE_CSV, index=False)

print(f"✅ Done: {OUTPUT_CSV}")
print(f"💾 Cache: {CACHE_CSV}")
print(f"✍️ Overrides file in use: {OVERRIDES_JSON}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.3 MB/s eta 0:00:00
🔧 Using overrides file: overrides_expanded.json
   Loaded overrides: 336 (normalized Arabic keys: 216)
   • لوز  ->  almonds
   • القرفة  ->  cinnamon
   • جوز الهند  ->  coconut
   • بيض  ->  egg
   • جبن  ->  cheese
   • خبز  ->  bread
   • طماطم  ->  tomato
   • فلفل أخضر حار  ->  green chili pepper
🧹 Purged 0 cache entries that now have overrides
📝 Translating columns: ['dish_name', 'classifications', 'image_file', 'scrape_date']
➡️  Translating: dish_name
➡️  Translating: classifications
➡️  Translating: image_file
➡️  Translating: scrape_date
✅ Done: SaudiFoodFile_english_FIXED.csv
💾 Cache: translation_cache.csv
✍️ Overrides file in use: overrides_expanded.json


#Step 2: Merge Image File names and Assign Unique IDs

**what is done**


We’re combining the Arabic and English datasets so that each English row keeps its original image file name and a new unique ID.

**Why:**

The Arabic file has the real image filenames (used later for matching or renaming images).

The English file has the cleaned and translated dish information.

Merging them lets us link each translated dish to its actual image and give every dish a clear numeric ID for easy reference in preprocessing and modeling.

**steps:**

Read both CSVs using pandas.

Find the correct “image file” column from the Arabic dataset.

Add that column into the English dataset as original names.

Create a new column image_id that numbers each row sequentially.

Save the final combined dataset as a new CSV for later steps.


**Input & Output for Step 1**



/content/SaudiFoodFile.csv → (Arabic dataset with original image filenames)

/content/SaudiFoodFile_english_FIXED.csv → (English translated dataset)

Output file:

/content/SaudiFoodFile_english_WITH_original_names_and_ID.csv → (English dataset with added original names and image_id columns)

In [3]:
# === Extract 'image file' -> add as 'original names' -> add image_id ===
import pandas as pd

# 1) Load both files (put both CSVs in /content first)
ar_path = "/content/SaudiFoodFile.csv"
en_path = "/content/SaudiFoodFile_english_FIXED.csv"

df_ar = pd.read_csv(ar_path, encoding="utf-8-sig")
df_en = pd.read_csv(en_path, encoding="utf-8-sig")

# 2) Find the 'image file' column in the Arabic CSV (robust to spaces/underscores/case)
def find_col(cols, target="image file"):
    t = target.lower().replace("_", " ").strip()
    for c in cols:
        if c.lower().replace("_", " ").strip() == t:
            return c
    return None

image_col = find_col(df_ar.columns, "image file")
if image_col is None:
    raise KeyError(
        "Could not find a column named 'image file' in SaudiFoodFile.csv. "
        f"Available columns: {list(df_ar.columns)}"
    )

# 3) Basic sanity check (same length)
if len(df_ar) != len(df_en):
    raise ValueError(f"Row count mismatch! Arabic={len(df_ar)}, English={len(df_en)}")

# 4) Add as 'original names'
df_en["original names"] = df_ar[image_col]

# 5) Add an ID for each 'original names' row (1..N)
df_en["image_id"] = range(1, len(df_en) + 1)

# 6) Save
out_path = "/content/SaudiFoodFile_english_WITH_original_names_and_ID.csv"
df_en.to_csv(out_path, index=False, encoding="utf-8-sig")

print("✅ Done! Saved to:", out_path)
df_en.head()


✅ Done! Saved to: /content/SaudiFoodFile_english_WITH_original_names_and_ID.csv


,dish_name,classifications,image_file,scrape_date,original names,image_id
0,Traditional Hijazi almond coffee,almonds | cinnamon | coconut,images/traditional_hejazi_almond_coffee.jpg,30-09-25,images/قهوة_اللوز_الحجازية_التقليدية.jpg,1
1,Shakeshawka Hijazi for the Saudi National Day,egg | cheese | bread,images/Shakshuka_Hejazia_for_Saudi_National_Da...,30-09-25,images/شكشوكة_حجازية_لليوم_الوطني_السعودي.jpg,2
2,Saudi meat kabsa and the authority of Dakus,tomato | green chili pepper | salt | cumin | b...,images/Saudi_meat_kabsa_and_dakous_salad.jpg,30-09-25,images/كبسة_اللحمة_السعودية_وسلطة_الدقوس.jpg,3
3,How to work,dates | cardamom | cinnamon | ginger | ghee | egg,images/how_to_work_the_college_of_Saudi Arabia...,30-09-25,images/طريقة_عمل_الكليجة_السعودية.jpg,4
4,Chicken Kabsa in the Saudi way,saffron | cardamom | cinnamon | mixed spices |...,images/Chicken_Kabsa_Saudi_Style.jpg,30-09-25,images/كبسة_الدجاج_على_الطريقة_السعودية.jpg,5


#Step 3: Remove Duplicate Rows by 'original names'


**what is done**


We’re cleaning the dataset by removing any duplicate rows that have the same **original image name**.

**Why:**

Some images might appear more than once (for example, the same dish uploaded multiple times). Duplicates can **distort analysis and model training**, so we keep only the first occurrence of each unique image.

**steps:**

1. Load the previous cleaned CSV file.
2. Detect the **"original names"** column (even if spacing or capitalization differs).
3. Use `drop_duplicates()` to remove repeated image entries.
4. Save the new deduplicated file as **`imagesID_no_duplicates.csv`**.


Input & Output for This Step

Input file:
/content/SaudiFoodFile_english_WITH_original_names_and_ID.csv
(the file created in Step 1)

Output file:
/content/imagesID_no_duplicates.csv
(same data, but with duplicate image names removed)


In [4]:
# === Remove Duplicate Rows by 'original names' ===
import pandas as pd

# 1) Load your CSV
df = pd.read_csv("/content/SaudiFoodFile_english_WITH_original_names_and_ID.csv", encoding="utf-8-sig")

# 2) Identify the correct column name (handles variations)
cols = [c.lower().replace("_", " ").strip() for c in df.columns]
if "original names" in cols:
    col_name = df.columns[cols.index("original names")]
else:
    raise KeyError(f"❌ Column 'original names' not found. Columns: {df.columns.tolist()}")

# 3) Drop duplicate rows based on that column (keep first)
df_clean = df.drop_duplicates(subset=[col_name], keep="first")

# 4) Save cleaned file
output_path = "/content/imagesID_no_duplicates.csv"
df_clean.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f"✅ Duplicates removed successfully!")
print(f"Original rows: {len(df)}")
print(f"Cleaned rows:  {len(df_clean)}")
print(f"📁 Saved as: {output_path}")


✅ Duplicates removed successfully!
Original rows: 285
Cleaned rows:  278
📁 Saved as: /content/imagesID_no_duplicates.csv


# Step 4: Rename ZIP Images Using CSV Mapping (Arabic-safe)

**what is done**


Rename images inside a ZIP to the format img(ID).ext using the CSV’s original names → image_id mapping.

**Why:**


Ensures consistent, model-friendly filenames and reliable linkage between each dish row and its image.


**steps:**

(Optional) Upload CSV and ZIP via Colab widgets (or use fixed paths).

Read CSV and build a robust, Unicode-safe lookup from original names → image_id.

Iterate ZIP files; for each matched filename, write it out as img(ID).ext to the output folder.

Zip the results and write a rename report (renamed vs. skipped).


**Inputs:**

CSV: /content/imagesID_no_duplicates.csv (must contain original names and image_id)

ZIP: /content/images.zip (original images)


**Outputs:**

Folder: /content/renamed_images/ (renamed files as img(ID).ext)

ZIP: /content/renamed_images.zip (packaged renamed images)

Report CSV: /content/rename_report.csv (status for each file)

In [8]:
# ===========================
# Rename images from ZIP using CSV (Arabic-safe)
# ===========================

# --- SETTINGS ---
USE_UPLOAD_WIDGETS = False  # True → show upload widgets for CSV + ZIP. False → use paths below.

CSV_PATH  = "/content/imagesID_no_duplicates.csv"  # used when USE_UPLOAD_WIDGETS=False
ZIP_PATH  = "/content/images.zip"                  # used when USE_UPLOAD_WIDGETS=False
OUT_DIR   = "/content/renamed_images"              # output folder
OUT_ZIP   = "/content/renamed_images.zip"          # zipped output
NAME_PREF = "img"                                  # final name format: img(ID).ext

import os, csv, zipfile, shutil, unicodedata, io
from pathlib import Path
import pandas as pd

# ---- (A) Upload files interactively in Colab (optional) ----
if USE_UPLOAD_WIDGETS:
    try:
        from google.colab import files
        print("📤 Please upload your CSV (with 'original names' + 'image_id'):")
        up1 = files.upload()
        CSV_PATH = "/content/" + list(up1.keys())[0]
        print("📤 Please upload your IMAGES ZIP:")
        up2 = files.upload()
        ZIP_PATH = "/content/" + list(up2.keys())[0]
    except Exception as e:
        raise RuntimeError("Colab file upload failed. Set USE_UPLOAD_WIDGETS=False to use paths.") from e

print("CSV_PATH:", CSV_PATH)
print("ZIP_PATH:", ZIP_PATH)

# ---- Helpers ----
def norm_header(s: str) -> str:
    return s.lower().replace("_", " ").strip()

def norm_key(name: str) -> str:
    """
    Normalize a filename for robust matching:
    - keep only basename (drop any 'images/...').
    - NFC Unicode normalization for Arabic.
    - casefold() for case-insensitive match.
    - strip spaces.
    """
    base = Path(str(name)).name
    return unicodedata.normalize("NFC", base).casefold().strip()

# ---- 1) Load mapping CSV (Arabic-safe) ----
df = pd.read_csv(CSV_PATH, encoding="utf-8-sig")
col_map = {norm_header(c): c for c in df.columns}

# find columns
if "original names" not in col_map:
    # try common variants
    for alt in ["original name", "image file", "image", "images", "original_images"]:
        if alt in col_map:
            col_map["original names"] = col_map[alt]
            break

id_col_key = "image id" if "image id" in col_map else ("image_id" if "image_id" in col_map else None)
if "original names" not in col_map or id_col_key is None:
    raise KeyError(f"CSV must have 'original names' and 'image_id' columns. Found: {list(df.columns)}")

orig_col = col_map["original names"]
id_col   = col_map[id_col_key]

# Build lookup: normalized basename → image_id (int)
lookup = {}
dups = df[orig_col].duplicated(keep=False)
if dups.any():
    print("⚠️ CSV contains duplicate values in 'original names'. First occurrence will be used.")

for _, row in df.iterrows():
    key = norm_key(row[orig_col])
    # keep first occurrence
    if key not in lookup:
        lookup[key] = int(row[id_col])

print(f"✅ Mapping loaded: {len(lookup)} names → IDs")

# ---- 2) Prepare output dir ----
shutil.rmtree(OUT_DIR, ignore_errors=True)
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

# ---- 3) Read ZIP and rename files ----
renamed, skipped = [], []

def write_member_to(path_out, zin, zinfo):
    with zin.open(zinfo) as src, open(path_out, "wb") as dst:
        dst.write(src.read())

with zipfile.ZipFile(ZIP_PATH, "r") as zin:
    for zinfo in zin.infolist():
        if zinfo.is_dir():
            continue

        # Get the basename (ignore any folders inside the zip)
        orig_zip_path = zinfo.filename
        base = Path(orig_zip_path).name
        key  = norm_key(base)

        if key in lookup:
            img_id = lookup[key]
            ext = Path(base).suffix  # keep original extension
            new_name = f"{NAME_PREF}({img_id}){ext}"
            out_path = os.path.join(OUT_DIR, new_name)
            write_member_to(out_path, zin, zinfo)
            renamed.append((orig_zip_path, new_name, img_id))
        else:
            skipped.append((orig_zip_path, "no match for 'original names'"))

# ---- 4) Zip the output folder ----
shutil.make_archive(OUT_ZIP.replace(".zip", ""), "zip", OUT_DIR)

# ---- 5) Report CSV ----
report_path = "/content/rename_report.csv"
with open(report_path, "w", newline="", encoding="utf-8-sig") as f:
    w = csv.writer(f)
    w.writerow(["status", "zip_original_path", "new_name_or_reason", "image_id"])
    for o, n, i in renamed:
        w.writerow(["renamed", o, n, i])
    for o, reason in skipped:
        w.writerow(["skipped", o, reason, ""])

print("🎉 Done!")
print(f"• Renamed: {len(renamed)}")
print(f"• Skipped (no match): {len(skipped)}")
print(f"📂 Output folder: {OUT_DIR}")
print(f"🗜️ Output zip:    {OUT_ZIP}")
print(f"🧾 Report CSV:    {report_path}")

CSV_PATH: /content/imagesID_no_duplicates.csv
ZIP_PATH: /content/images.zip
✅ Mapping loaded: 278 names → IDs
🎉 Done!
• Renamed: 278
• Skipped (no match): 55
📂 Output folder: /content/renamed_images
🗜️ Output zip:    /content/renamed_images.zip
🧾 Report CSV:    /content/rename_report.csv


#Step 5: Update Image Filenames Inside the CSV

**what is done**


Rename the image file entries inside the CSV so they match the renamed image format img(ID).ext.


**Why:**
To keep the dataset consistent with the renamed images in your folder/ZIP — ensuring every row’s filename matches its actual image file.


**steps:**

Load the latest CSV containing image_id.

Automatically find the column that holds image filenames.

Replace each filename with the new standardized format img(ID).ext while keeping the same extension.

Save the updated CSV for use in later steps.



**Input file:**
/content/imagesID_no_duplicates.csv (before renaming inside CSV)

**Output file:**
/content/imagesID_renamed_in_csv.csv (filenames now follow the img(ID).ext format)

In [9]:
# === Rename filenames in CSV to match img(ID).ext format ===
import pandas as pd
from pathlib import Path

# 1) Load your CSV (make sure it contains 'image_id' and the column with the image file paths)
CSV_PATH = "/content/imagesID_no_duplicates.csv"  # or your latest version
df = pd.read_csv(CSV_PATH, encoding="utf-8-sig")

# 2) Identify the image path column (commonly 'image_file' or 'images')
def find_col(cols, target="image"):
    t = target.lower()
    for c in cols:
        if t in c.lower():
            return c
    return None

img_col = find_col(df.columns, "image")
if not img_col:
    raise KeyError(f"No column found containing 'image'. Columns: {df.columns.tolist()}")

# 3) Rename each image according to its ID, keeping the same file extension
def make_new_name(old_path, image_id):
    ext = Path(str(old_path)).suffix or ".jpg"  # default to .jpg if missing
    return f"img({image_id}){ext}"

df[img_col] = [make_new_name(df.loc[i, img_col], df.loc[i, "image_id"]) for i in range(len(df))]

# 4) Save the updated CSV
OUTPUT_PATH = "/content/imagesID_renamed_in_csv.csv"
df.to_csv(OUTPUT_PATH, index=False, encoding="utf-8-sig")

print("✅ All image filenames updated inside the CSV.")
print(f"📁 Saved as: {OUTPUT_PATH}")
df.head()


✅ All image filenames updated inside the CSV.
📁 Saved as: /content/imagesID_renamed_in_csv.csv


,dish_name,classifications,image_file,scrape_date,original names,image_id
0,Traditional Hijazi almond coffee,almonds | cinnamon | coconut,img(1).jpg,30-09-25,images/قهوة_اللوز_الحجازية_التقليدية.jpg,1
1,Shakeshawka Hijazi for the Saudi National Day,egg | cheese | bread,img(2).jpg,30-09-25,images/شكشوكة_حجازية_لليوم_الوطني_السعودي.jpg,2
2,Saudi meat kabsa and the authority of Dakus,tomato | green chili pepper | salt | cumin | b...,img(3).jpg,30-09-25,images/كبسة_اللحمة_السعودية_وسلطة_الدقوس.jpg,3
3,How to work,dates | cardamom | cinnamon | ginger | ghee | egg,img(4).jpg,30-09-25,images/طريقة_عمل_الكليجة_السعودية.jpg,4
4,Chicken Kabsa in the Saudi way,saffron | cardamom | cinnamon | mixed spices |...,img(5).jpg,30-09-25,images/كبسة_الدجاج_على_الطريقة_السعودية.jpg,5


#Step 6: Remove the “original names” Column


Delete the **`original names`** column from the dataset.

**Why:**
After all images have been renamed and matched using IDs, the original filenames are no longer needed.
Removing this column keeps the dataset **clean and ready for modeling or analysis**.

**steps**

1. Load the latest CSV file.
2. Find and drop the **`original names`** column (handles different spellings or spacing).
3. Save the final cleaned version.



**Input file:**
`/content/imagesID_renamed_in_csv.csv`

**Output file:**
`/content/remove_original_names.csv` *(final cleaned dataset without the old image name column)*


In [10]:
# === Remove "original names" column ===
import pandas as pd

# 1) Load your CSV file
df = pd.read_csv("/content/imagesID_renamed_in_csv.csv", encoding="utf-8-sig")

# 2) Remove the column safely (handles naming variations)
cols = [c.lower().replace("_", " ").strip() for c in df.columns]
if "original names" in cols:
    col_name = df.columns[cols.index("original names")]
    df = df.drop(columns=[col_name])
else:
    raise KeyError(f"Column 'original names' not found. Columns: {df.columns.tolist()}")

# 3) Save the new file
output_path = "/content/remove_original_names.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")

print("✅ 'original names' column removed.")
print(f"📁 Clean file saved as: {output_path}")


✅ 'original names' column removed.
📁 Clean file saved as: /content/remove_original_names.csv


#Step 7: Clean and Standardize Dish Names

**what is done**

This step cleans and unifies all dish names — keeping authentic Arabic and Middle Eastern dishes intact while removing unnecessary English words, event mentions, and quantity descriptions.

**Why:**
To make the dataset consistent and ready for analysis by:

Removing irrelevant phrases like “How to make” or “for Saudi National Day”.

Avoiding over-cleaning that might erase Arabic or culturally important dish names.

Standardizing different spellings (e.g., kbsa, kabsah → Kabsa) into one consistent form.

**steps:**

Load the latest dataset (remove_original_names.csv).

Use clean_dish_name() to strip only specific English words, numbers, and phrases — preserving Arabic names.

Use standardize_dish_name() to unify variations and detect dish types and proteins (e.g., Kabsa Chicken).

Display before/after examples of cleaned names.

Save the cleaned and standardized dataset as SaudiFoodFile_cc.csv.

**Input file:**
remove_original_names.csv

**Output file:**
SaudiFoodFile_cleaned_dishName.csv (final dataset with standardized and culturally accurate dish names)

In [11]:
import pandas as pd
import numpy as np
import re

# Load the data
df = pd.read_csv('remove_original_names.csv')

# Display initial data info
print("Initial data shape:", df.shape)
print("\nFirst few rows:")
print(df.head())

# Task 1: Clean dish names - remove descriptions, quantities, and non-ingredient information
def clean_dish_name(name):
    """
    Remove descriptions, quantities, and non-ingredient information from dish names
    while preserving authentic Arabic/Middle Eastern dish names
    """
    # Common patterns to remove (ONLY specific English descriptions)
    patterns_to_remove = [
        # Occasions and events (English)
        r'\bfor saudi national day\b',
        r'\bfor hosting\b',
        r'\bsummer offer\b',

        # Cooking methods and styles (English)
        r'\bhow to make\b',
        r'\bhow to boil\b',
        r'\bsaudi style\b',
        r'\bhijazi style\b',
        r'\btraditional\b',
        r'\bauthentic\b',
        r'\beasy\b',
        r'\bcopycat recipe\b',
        r'\bslow-?roast\b',
        r'\bno bake\b',
        r'\brussian style\b',
        r'\brussian\b',

        # Remove ALL parentheses and their content
        r'\([^)]*\)',

        # Very specific quantity descriptions only
        r'\bwhole grain\b',
        r'\bhalf a piece\b',
        r'\bhalf piece\b',
        r'\bone person\b',

        # General English descriptions (be very specific)
        r'\bmethod for\b',
        r'\baccording to\b',
        r'\bway\b',
        r'\brecipe\b',
    ]

    cleaned_name = name.strip()

    # Remove only very specific patterns
    for pattern in patterns_to_remove:
        cleaned_name = re.sub(pattern, '', cleaned_name, flags=re.IGNORECASE)

    # Clean up extra spaces and punctuation carefully
    cleaned_name = re.sub(r'\s+', ' ', cleaned_name)
    cleaned_name = cleaned_name.strip()

    return cleaned_name

# Task 2: Standardize dish name variations to consistent naming
def standardize_dish_name(name):
    """
    Standardize variations of dish names to consistent naming conventions
    while preserving the main dish identity
    """
    if not name or name.strip() == '':
        return 'Unclassified Dish'

    # First, extract the main dish components
    name_lower = name.lower()

    # Identify the main dish type
    main_dish = None
    protein = None

    # Check for main dishes (preserve Arabic names)
    dish_patterns = {
        'Kabsa': r'\bkabsa\b|\bkabsah\b|\bkbsa\b',
        'Mandi': r'\bmandi\b',
        'Madhbi': r'\bmadhbi\b',
        'Madfoon': r'\bmadfoon\b',
        'Madghut': r'\bmadghog\b|\bmadjoun\b|\bmadjoon\b',
        'Shakshuka': r'\bshakshuka\b|\bshaksuka\b',
        'Jareesh': r'\bjareesh\b|\bjarish\b|\bgroats\b',
        'Maqluba': r'\bmaqluba\b|\bmakloubeh\b|\bmagloba\b',
        'Kleja': r'\bkleija\b|\bkleja\b|\bklija\b',
        'Maamoul': r'\bmaamoul\b|\bmamoul\b',
        'Mutabak': r'\bmutabbaq\b|\bmutabak\b',
        'Sambusa': r'\bsambosa\b|\bsambousek\b|\bsamosa\b',
        'Basbousa': r'\bbasbousa\b|\bbasbosa\b',
        'Kunafa': r'\bkunafa\b|\bknafeh\b',
        'Mulukhiyah': r'\bmulukhiyah\b|\bmolokhia\b|\bmolokhiya\b',
        'Saleek': r'\bsaleek\b|\bsaleeq\b|\bsuliq\b|\bsulait\b',
        'Freekeh': r'\bfreekeh\b|\bfreekey\b',
        'Mujaddara': r'\bmujadara\b|\bmujaddara\b',
        'Luqaimat': r'\bluqaimat\b|\bluqaymat\b',
        'Harees': r'\bharees\b|\bhareeseh\b',
        'Thareed': r'\bthareed\b|\btharid\b',
    }

    for dish, pattern in dish_patterns.items():
        if re.search(pattern, name_lower):
            main_dish = dish
            break

    # If no specific dish found, try to identify by main components
    if not main_dish:
        # Check for proteins
        if re.search(r'\bchicken\b', name_lower):
            protein = 'Chicken'
        elif re.search(r'\blamb\b', name_lower):
            protein = 'Lamb'
        elif re.search(r'\bmeat\b|\bbeef\b', name_lower):
            protein = 'Meat'
        elif re.search(r'\bfish\b', name_lower):
            protein = 'Fish'
        elif re.search(r'\bshrimp\b', name_lower):
            protein = 'Shrimp'
        elif re.search(r'\bcamel\b', name_lower):
            protein = 'Camel'

    # Build standardized name
    if main_dish:
        if protein:
            standardized_name = f"{main_dish} {protein}"
        else:
            standardized_name = main_dish
    elif protein:
        # If we only have protein but no specific dish, use the cleaned name
        standardized_name = name.title()
    else:
        # For names without clear dish type, clean but preserve the name
        standardized_name = name.title()

        # Apply gentle standardization for common variations
        variations = {
            r'\bkabsah\b': 'Kabsa',
            r'\bkbsa\b': 'Kabsa',
            r'\bmandi\b': 'Mandi',
            r'\bmutabbaq\b': 'Mutabak',
            r'\bsambosa\b': 'Sambusa',
            r'\bmaamoul\b': 'Maamoul',
            r'\bkunafa\b': 'Kunafa',
            r'\bbasbousa\b': 'Basbousa',
        }

        for pattern, replacement in variations.items():
            standardized_name = re.sub(pattern, replacement, standardized_name, flags=re.IGNORECASE)

    return standardized_name.strip()

# Apply cleaning and standardization
print("\nApplying data cleaning...")

# Create cleaned dish names
df['cleaned_dish_name'] = df['dish_name'].apply(clean_dish_name)
df['standardized_dish_name'] = df['cleaned_dish_name'].apply(standardize_dish_name)

# Show before and after examples - focus on problem cases
print("\nName cleaning examples (focusing on problem cases):")
problem_cases = [
    "A quarter of mandi chicken",
    "Quarter goat",
    "Chicken kabsa with rice",
    "Meat kabsa and daqoos salad",
    "Saudi meat kabsa and daqoos salad",
    "How to make Saudi kleija",
    "Saudi style chicken kabsa",
    "Russian style borscht soup",
    "Chicken Kabsa (curry) with rice",
    "Plain mandi rice"
]

for i, row in df.iterrows():
    if any(case.lower() in row['dish_name'].lower() for case in problem_cases):
        print(f"Original: {row['dish_name']}")
        print(f"Cleaned: {row['cleaned_dish_name']}")
        print(f"Standardized: {row['standardized_dish_name']}")
        print("-" * 50)

# Show most common dish names after standardization
print("\nMost common standardized dish names:")
print(df['standardized_dish_name'].value_counts().head(20))

# Save the cleaned data
df_cleaned = df.copy()
# Keep original name and use standardized as main dish name
df_cleaned['dish_name_original'] = df['dish_name']
df_cleaned['dish_name'] = df['standardized_dish_name']

# Drop temporary columns
df_cleaned = df_cleaned.drop(['cleaned_dish_name', 'standardized_dish_name'], axis=1)

print(f"\nFinal data shape: {df_cleaned.shape}")
print("\nFirst few rows of cleaned data:")
print(df_cleaned[['dish_name_original', 'dish_name']].head(20))

# Save to new CSV file
output_filename = 'SaudiFoodFile_cleaned_dishName.csv'
df_cleaned.to_csv(output_filename, index=False)
print(f"\nCleaned data saved to: {output_filename}")

# Additional analysis: Show name standardization results
print("\n" + "="*80)
print("NAME STANDARDIZATION SUMMARY")
print("="*80)

# Show specific problem cases and their resolution
print("\nSpecific problem cases and their resolution:")
test_cases = [
    "A quarter of mandi chicken",
    "Quarter goat",
    "Chicken kabsa with rice",
    "Saudi meat kabsa and daqoos salad",
    "How to make Saudi kleija",
    "Saudi style chicken kabsa",
    "Plain mandi rice",
    "Half grilled chicken with rice",
    "Russian style borscht soup",
    "Chicken Kabsa (curry) with rice",
    "Mandi Chicken (whole grain)",
    "Grilled chicken (half a piece)"
]

for case in test_cases:
    cleaned = clean_dish_name(case)
    standardized = standardize_dish_name(cleaned)
    print(f"Original: '{case}'")
    print(f"Cleaned: '{cleaned}'")
    print(f"Standardized: '{standardized}'")
    print("-" * 40)

# Show reduction in unique names
original_unique = df['dish_name'].nunique()
cleaned_unique = df_cleaned['dish_name'].nunique()
print(f"\nUnique name reduction: {original_unique} → {cleaned_unique} ({(1-cleaned_unique/original_unique)*100:.1f}% reduction)")

print("\nCleaning complete! Arabic dish names are preserved while English descriptions are removed.")

Initial data shape: (278, 5)

First few rows:
                                       dish_name  \
0               Traditional Hijazi almond coffee   
1  Shakeshawka Hijazi for the Saudi National Day   
2    Saudi meat kabsa and the authority of Dakus   
3                                    How to work   
4                 Chicken Kabsa in the Saudi way   

                                     classifications  image_file scrape_date  \
0                       almonds | cinnamon | coconut  img(1).jpg    30-09-25   
1                               egg | cheese | bread  img(2).jpg    30-09-25   
2  tomato | green chili pepper | salt | cumin | b...  img(3).jpg    30-09-25   
3  dates | cardamom | cinnamon | ginger | ghee | egg  img(4).jpg    30-09-25   
4  saffron | cardamom | cinnamon | mixed spices |...  img(5).jpg    30-09-25   

   image_id  
0         1  
1         2  
2         3  
3         4  
4         5  

Applying data cleaning...

Name cleaning examples (focusing on problem case

#Step 7: Clean and Standardize Ingredients


This step cleans and standardizes all ingredient information before the consistency check.
Some dishes originally had messy or incomplete ingredient lists — for example:

> `"olive oil / tomatoes - onion, chilli"` or sometimes just `"unknown"`

The goal here is to convert everything into clear, structured ingredient lists such as:

> `["olive oil", "tomato", "onion", "chili"]`
> and to replace unclear or missing entries like `["unknown"]` with the clear label `["unknown_ingredients"]`.

**What this step does:**

1. **Splits ingredients correctly:**
   Separates text using real separators (`|`, `/`, `,`, `;`, Arabic commas, or dashes) without breaking multi-word names.

2. **Protects multi-word ingredients:**
   Keeps terms like *olive oil* or *tomato paste* together as one ingredient.

3. **Removes non-ingredient words:**
   Drops extra words such as *add*, *garnish*, *with*, or *hot* that aren’t actual ingredients.

4. **Normalizes plurals and spellings:**
   Converts plurals (*tomatoes → tomato*) and unifies spellings (*chilli*, *chilies* → *chili*).

5. **Handles unknown or missing entries:**
   Any ingredient cell that is empty or simply says `"not found"` or `"unknown"` becomes `["unknown"]` to keep the format consistent.

6. **Embedding + Clustering (semantic cleaning):**
   group similar ingredient names (like *cardamon* and *cardamom*) into one canonical form.

7. **Creates standardized ingredient lists:**
   Every dish ends up with a clean list of consistent, machine-readable ingredients.

**Outputs:**

* `SaudiFoodFile_standardized.csv` → the final cleaned ingredient lists per dish. will be used in the next step.
* `ingredient_clusters_report.csv` → groups of similar ingredients and their canonical names (helper)
* `ingredient_canonical_map.json` → mapping of each ingredient to its canonical form (helper)


In [12]:

# ============================================================
#  — Ingredient Cleaning
# - Split on real separators only (| / \ , ; ، - with spaces, bullets)
# - Protect multi-word ingredient phrases (KEEP_PHRASES)
# - Extract ingredients from stray sentences; drop non-ingredient words
# - Canonicalize with ALIASES + singularization + fuzzy nudge
# - Output only: dish_name | classifications_std_list | image_file | scrape_date
# ============================================================

!pip install -q pandas sentence-transformers scikit-learn rapidfuzz inflect

import re, json, pandas as pd
from collections import Counter, defaultdict
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
from rapidfuzz import process, fuzz
import inflect

# --- Unknown/placeholder detection ---
UNKNOWN_PATTERNS = [
    r"^\s*not\s*found\s*$",
    r"^\s*unknown\s*$",
    r"^\s*n/?a\s*$",
    r"^\s*none\s*$",
    r"^\s*null\s*$",
    r"^\s*missing\s*$",
    r"^\s*بدون\s*$",           # Arabic: without
    r"^\s*غير\s*متوفر\s*$",   # Arabic: unavailable
]

UNKNOWN_REGEXES = [re.compile(p, re.IGNORECASE) for p in UNKNOWN_PATTERNS]

def is_unknown_text(s: str) -> bool:
    return any(rx.match(s) for rx in UNKNOWN_REGEXES)


# ----------------------------
# Config
# ----------------------------
INPUT_CSV  = "SaudiFoodFile_cleaned_dishName.csv"
OUTPUT_CSV = "SaudiFoodFile_standardized.csv"
REPORT_CSV = "ingredient_clusters_report.csv"
MAP_JSON   = "ingredient_canonical_map.json"

CLASS_COL_CANDIDATES = ["classifications", "classifications_en", "classfications"]
MODEL_NAME = "sentence-transformers/paraphrase-MiniLM-L6-v2"
DISTANCE_THRESHOLD = 0.35
FUZZY_SCORE_CUTOFF = 92

# ----------------------------
# Phrase protection & vocabulary
# ----------------------------
# Multi-word ingredients to KEEP as single tokens
KEEP_PHRASES = {
    # oils / dairy
    "olive oil", "vegetable oil", "clarified butter", "milk powder", "butter milk", "heavy cream",
    # stocks / sauces / pastes
    "tomato paste", "tomato sauce", "soy sauce", "pomegranate molasses", "date molasses", "rose water", "orange blossom water",
    # veg & herbs
    "bell pepper", "green onion", "spring onion", "bay leaves", "mint leaves", "parsley leaves", "coriander leaves",
    # spices
    "black lime", "mixed spices", "spice mix", "allspice", "black pepper", "white pepper",
    # proteins / grains
    "basmati rice", "chicken stock", "beef stock", "vegetable stock",
}

# Synonyms/variants → canonical (lowercase)
ALIASES = {
    # souring / lime
    "dried lime": "black lime", "dried limes": "black lime", "omani lime": "black lime",
    "omani limes": "black lime", "loomi": "black lime", "black lemon": "black lime",

    # oils/fats/dairy
    "veg oil": "vegetable oil", "olive oils": "olive oil", "butter milk": "buttermilk",
    "yoghurt": "yogurt", "labnah": "labneh",

    # herbs & veg
    "cilantro": "coriander", "coriander leaves": "coriander", "green coriander": "coriander",
    "parsley leaves": "parsley", "mint leaves": "mint", "spring onion": "green onion",

    # peppers
    "capsicum": "bell pepper", "green pepper": "bell pepper", "sweet pepper": "bell pepper",
    "chilli": "chili", "chilies": "chili", "chillies": "chili", "chili pepper": "chili", "chili peppers": "chili",

    # powders & sticks -> base spice
    "turmeric powder": "turmeric", "ginger powder": "ginger",
    "garlic powder": "garlic", "onion powder": "onion",
    "cinnamon stick": "cinnamon", "cinnamon sticks": "cinnamon",
    "cardamon": "cardamom",

    # sauces/pastes/syrups
    "tomato purée": "tomato paste", "tomato puree": "tomato paste",
    "simple syrup": "sugar syrup",

    # pulses/grains
    "garbanzo beans": "chickpeas", "garbanzo bean": "chickpeas",
    "chick peas": "chickpeas", "chick pea": "chickpeas",
    "black eyed beans": "black-eyed beans", "black eyed pea": "black-eyed beans",

    # spice mixes / generic
    "spice mix": "mixed spices", "mix spices": "mixed spices", "spices mix": "mixed spices"
}

# Words that are NOT ingredients (filler, verbs, instructions)
NON_ING_WORDS = {
    "after","decorate","decoration","decorations","add","with","such","touch","patriotic","cream",
    "or","and","the","a","an","then","until","when","like","as","to","for","of","into","over",
    "warm","hot","cold","slice","sliced","diced","chopped","minced","ground","crushed","whole",
    "fresh","optional","needed","garnish","make","prepare","preparation","cook","cooked","baked",
    "boiled","fried","seauted","sauteed","browned","mix","mixed","topping","kitchen","precise","instant"
}

# ----------------------------
# Helpers
# ----------------------------
SEP_NORMALIZER = re.compile(r"[|/\\،;,]+")      # | / \ , ; Arabic comma
AROUND_HYPHEN  = re.compile(r"\s*[-–—]\s*")     # spaced hyphens/dashes as separators
BULLETS        = re.compile(r"[•·]+")

# After splitting, strip these inside tokens
PUNCT_DROP_INSIDE = re.compile(r"[-_/\\|]+")
NONWORD           = re.compile(r"[^\w\s\(\)]")

IRREGULAR = {"tomatoes":"tomato","potatoes":"potato","limes":"lime","chillies":"chili","chilies":"chili","cloves":"cloves"}
p = inflect.engine()

def safe_str(x):
    if x is None: return ""
    s = str(x)
    return "" if s.strip().lower() in {"", "nan", "none", "null"} else s

def protect_phrases(text: str) -> str:
    """Replace spaces inside KEEP_PHRASES with underscores to protect them."""
    s = text
    # longer phrases first to avoid partial overlaps
    for ph in sorted(KEEP_PHRASES, key=lambda x: -len(x)):
        pattern = r"\b" + re.escape(ph) + r"\b"
        s = re.sub(pattern, ph.replace(" ", "_"), s, flags=re.IGNORECASE)
    return s

def normalize_separators(s: str) -> str:
    s = BULLETS.sub("|", s)
    s = AROUND_HYPHEN.sub("|", s)           # ' - ' → '|'
    s = SEP_NORMALIZER.sub("|", s)          # unify to '|'
    s = re.sub(r"\|{2,}", "|", s).strip("| ")
    return s

def to_singular(word: str) -> str:
    w = word.strip().lower()
    if w in IRREGULAR: return IRREGULAR[w]
    s = p.singular_noun(w)
    return s if isinstance(s, str) and s else w

def clean_phrase(s: str) -> str:
    s = s.strip().lower()
    s = PUNCT_DROP_INSIDE.sub(" ", s)
    s = NONWORD.sub(" ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def drop_non_ingredients(words):
    return [w for w in words if w not in NON_ING_WORDS and len(w) > 1]

def split_to_tokens(cell) -> list[str]:
    """
    - If the whole cell is a placeholder (not found/unknown/etc.), return []
      so it becomes ["unknown_ingredients"] later.
    - Normalize separators to '|', protect phrases, split on '|'.
    - Extract protected phrases, then ingredient-like words only.
    """
    raw = safe_str(cell)
    if not raw:
        return []
    # whole-cell unknowns → empty list (will become ["unknown_ingredients"])
    if is_unknown_text(raw):
        return []

    # normalize separators and protect phrases
    s = normalize_separators(raw)
    s = protect_phrases(s)
    parts = [p.strip() for p in s.split("|") if p.strip()]

    tokens = []
    for p in parts:
        # per-part unknowns too
        if is_unknown_text(p):
            # skip this part entirely
            continue

        # restore underscores for already isolated protected phrases
        if "_" in p and p in KEEP_PHRASES or p.replace("_"," ") in KEEP_PHRASES:
            tokens.append(p.replace("_"," "))
            continue

        # 1) collect any protected phrases still inside
        found = []
        rest  = p
        for ph in sorted(KEEP_PHRASES, key=lambda x: -len(x)):
            ph_prot = ph.replace(" ", "_")
            if ph_prot.lower() in rest.lower():
                found.append(ph)
                rest = re.sub(re.escape(ph_prot), " ", rest, flags=re.IGNORECASE)

        # 2) remaining words (filtered)
        rest = clean_phrase(rest.replace("_"," "))
        words = drop_non_ingredients(rest.split())
        words = [to_singular(w) for w in words]

        for ph in found:
            tokens.append(ph)
        for w in words:
            tokens.append(w)

    # alias + dedupe
    out = []
    for t in tokens:
        t0 = ALIASES.get(t.strip().lower().replace("_"," "), t.strip().lower().replace("_"," "))
        if t0:
            out.append(t0)

    seen, dedup = set(), []
    for x in out:
        if x not in seen:
            dedup.append(x); seen.add(x)
    return dedup

    # normalize separators and protect phrases
    s = normalize_separators(raw)
    s = protect_phrases(s)
    parts = [p.strip() for p in s.split("|") if p.strip()]

    tokens = []
    for p in parts:
        # restore underscores for already isolated protected phrases
        if "_" in p and p in KEEP_PHRASES or p.replace("_"," ") in KEEP_PHRASES:
            tokens.append(p.replace("_"," "))
            continue

        # If it's a sentence: extract any protected phrases inside, then words
        # 1) pull out protected phrases inside the piece
        found = []
        rest  = p
        for ph in sorted(KEEP_PHRASES, key=lambda x: -len(x)):
            ph_prot = ph.replace(" ", "_")
            if ph_prot.lower() in rest.lower():
                # collect and remove
                found.append(ph)
                rest = re.sub(re.escape(ph_prot), " ", rest, flags=re.IGNORECASE)

        # 2) remaining words
        rest = clean_phrase(rest.replace("_"," "))
        words = drop_non_ingredients(rest.split())
        # singularize last token of any 1-2 word units (light touch)
        words = [to_singular(w) for w in words]

        # combine: protected phrases + remaining words
        for ph in found:
            tokens.append(ph)
        for w in words:
            tokens.append(w)

    # apply aliases & final cleanup
    out = []
    for t in tokens:
        t0 = t.strip().lower()
        t0 = t0.replace("_"," ")
        if not t0:
            continue
        t0 = ALIASES.get(t0, t0)
        out.append(t0)

    # de-dup preserve order
    seen, dedup = set(), []
    for x in out:
        if x not in seen:
            dedup.append(x); seen.add(x)

    return dedup

def normalize_for_clustering(token: str) -> str:
    """Secondary normalization for clustering stage."""
    t = token.strip().lower()
    t = re.sub(r"\s+", " ", t)
    return t

# ----------------------------
# Load data (robust to "NA")
# ----------------------------
df = pd.read_csv(INPUT_CSV, encoding="utf-8", keep_default_na=False, na_filter=False)
n_rows = len(df)
CLASS_COL = next((c for c in CLASS_COL_CANDIDATES if c in df.columns), None)
if not CLASS_COL:
    raise ValueError(f"Could not find classifications column. Found: {list(df.columns)}")

# ----------------------------
# Tokenize all rows with phrase-aware extractor
# ----------------------------
raw_lists = df[CLASS_COL].apply(split_to_tokens)

# Clean tokens for clustering
cleaned_all, bag = [], []
for tokens in raw_lists:
    cleaned = []
    for t in tokens:
        nt = normalize_for_clustering(t)
        if nt:
            cleaned.append(nt)
            bag.append(nt)
    cleaned_all.append(cleaned)

freq = Counter(bag)
unique_tokens = list(freq.keys())

# If no tokens → fill unknowns but still emit rows
if not unique_tokens:
    df["classifications_std_list"] = [["unknown_ingredients"] for _ in range(n_rows)]
    out = df[["dish_name","classifications_std_list","image_file","scrape_date"]]
    out.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")
    print(f"✅ Done (all unknown). Rows: {len(out)}")
    raise SystemExit

# ----------------------------
# Embed & cluster
# ----------------------------
model = SentenceTransformer(MODEL_NAME)
emb = model.encode(unique_tokens, show_progress_bar=True, normalize_embeddings=True)

try:
    clust = AgglomerativeClustering(
        n_clusters=None, linkage="average",
        metric="cosine", distance_threshold=DISTANCE_THRESHOLD
    )
except TypeError:
    clust = AgglomerativeClustering(
        n_clusters=None, linkage="average",
        affinity="cosine", distance_threshold=DISTANCE_THRESHOLD
    )
labels = clust.fit_predict(emb)

clusters = defaultdict(list)
for tok, lab in zip(unique_tokens, labels):
    clusters[lab].append(tok)

# canonical per cluster: most frequent (tie -> shortest)
cluster_canonical = {lab: sorted(toks, key=lambda t: (-freq[t], len(t)))[0] for lab, toks in clusters.items()}
token2canon = {tok: cluster_canonical[lab] for tok, lab in zip(unique_tokens, labels)}

# optional fuzzy nudge
canonical_vocab = list(set(token2canon.values()))
def fuzzy_canon(token: str, cutoff=FUZZY_SCORE_CUTOFF):
    best = process.extractOne(token, canonical_vocab, scorer=fuzz.WRatio, score_cutoff=cutoff)
    return best[0] if best else token
for tok in list(token2canon.keys()):
    cand = fuzzy_canon(tok)
    if cand != token2canon[tok] and freq.get(cand, 0) >= freq.get(token2canon[tok], 0):
        token2canon[tok] = cand

# ----------------------------
# Apply mapping to EVERY row
# ----------------------------
std_lists = []
for cleaned in cleaned_all:
    mapped = [token2canon.get(t, t) for t in cleaned]
    # de-dup preserve order
    seen, dedup = set(), []
    for x in mapped:
        if x not in seen:
            dedup.append(x); seen.add(x)
    if not dedup:
        dedup = ["unknown_ingredients"]
    std_lists.append(dedup)

assert len(std_lists) == n_rows
df["classifications_std_list"] = std_lists

# --- Force unknown_ingredients for placeholder fragments like ["not","found"] ---
def coalesce_unknown(lst):
    # If list is empty, we'll handle later; if it exactly equals ["not","found"], force unknown
    if not lst:
        return ["unknown_ingredients"]
    lf = [x.strip().lower() for x in lst]
    if lf == ["not","found"] or lf == ["unknown"]:
        return ["unknown_ingredients"]
    # If list contains only non-ingredient placeholders, collapse too
    joined = " ".join(lf)
    if is_unknown_text(joined):
        return ["unknown_ingredients"]
    return lst

df["classifications_std_list"] = df["classifications_std_list"].apply(coalesce_unknown)


# ---------------------------------------------------------
# Post-processing consistency fix: collapse generic + specific variants
# ---------------------------------------------------------
canonical_collapse = {
    # generic → preferred
    "oil": "olive oil",
    "vegetable oil": "olive oil",
    "ghee": "butter",
    "yogurt": "labneh",       # example, if you prefer labneh
    # add any others you notice
}

def collapse_variants(lst):
    """Replace generic tokens with canonical equivalents and de-duplicate."""
    out = []
    seen = set()
    for x in lst:
        y = canonical_collapse.get(x, x)
        if y not in seen:
            out.append(y)
            seen.add(y)
    return out

df["classifications_std_list"] = df["classifications_std_list"].apply(collapse_variants)



# ----------------------------
# Reports & mapping
# ----------------------------
rows = []
for lab, toks in clusters.items():
    can = cluster_canonical[lab]
    for t in sorted(toks):
        rows.append({"cluster_id": lab, "canonical": can, "member": t, "member_freq": freq[t]})
pd.DataFrame(rows).sort_values(["canonical","member"]).to_csv(REPORT_CSV, index=False, encoding="utf-8")
with open(MAP_JSON, "w", encoding="utf-8") as f:
    json.dump(token2canon, f, ensure_ascii=False, indent=2)

# ----------------------------
# Save ONLY the requested 4 columns
# ----------------------------
required_cols = ["dish_name", "image_file", "scrape_date"]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing expected columns: {missing}")

out = df[["dish_name","classifications_std_list","image_file","scrape_date"]]
out.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")

print("✅ Done.")
print(f"Rows in/out: {n_rows} / {len(out)}")
print(f"Example rows:\n{out.head(6).to_string(index=False)}")




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 35.1 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Done.
Rows in/out: 278 / 278
Example rows:
                                    dish_name                                                                                                                                               classifications_std_list image_file scrape_date
                         Hijazi Almond Coffee                                                                                                                                            [almond, cinnamon, coconut] img(1).jpg    30-09-25
Shakeshawka Hijazi For The Saudi National Day                                                                                                                                                   [egg, cheese, bread] img(2).jpg    30-09-25
                                        Kabsa                                                                                                     [tomato, green, chili, pepper, salt, cumin, lemon, juice, vinegar] img(3).jpg    30-09-25
           

### **Ingredient Cleaning - Step 3 — Final Consistency Pass**

After cleaning and standardizing the ingredients, some names were still not completely consistent.
For example, a few dishes still used slightly different spellings or duplicate ingredient terms such as:

> ["paper", "chilli", "olive oils"] instead of ["chili", "olive oil"].

This step was added to double-check and correct any remaining inconsistencies in spelling, wording, or duplicates.
It reviews every ingredient list and makes final adjustments to ensure that all rows follow the same standard format.

In [13]:
# ============================================================
# Consistency pass over the produced CSV
# - Enforce canonical spellings & synonyms (e.g., chilli → chili)
# - Your requests: paper → chili, allspices → mixed spices
# - Remove non-ingredient words, dedupe, keep only 4 columns
# Outputs:
#   • SaudiFoodFile_standardized_consistent.csv
#   • consistency_changes_report.csv (what changed)
# ============================================================

import ast, re, json, pandas as pd

INPUT_CSV  = "SaudiFoodFile_standardized.csv"   # <— your produced file
OUTPUT_CSV = "SaudiFoodFile_standardized_consistent.csv"
REPORT_CSV = "consistency_changes_report.csv"

# If your column name differs, adjust here:
CLASS_COL = "classifications_std_list"
REQ_COLS  = ["dish_name", CLASS_COL, "image_file", "scrape_date"]

# ---------- canonicalization config ----------
# ✅ Add/adjust anything you want here:
ALIASES = {
    # your explicit requests
    "paper": "chili",              # e.g., OCR/typo -> chili
    "allspices": "mixed spices",
    "all-spices": "mixed spices",
    "all spice": "mixed spices",
    "all-spice": "mixed spices",
    "allspice": "mixed spices",    # if you prefer "mixed spices" as canonical

    # common spellings / plurals / variants
    "chilli": "chili",
    "chilies": "chili",
    "chillies": "chili",
    "green chilli": "chili",
    "green chili": "chili",
    "red chili": "chili",
    "red chilli": "chili",
    "chili pepper": "chili",
    "chili peppers": "chili",

    "black lemon": "black lime",
    "dried lime": "black lime",
    "dried limes": "black lime",
    "omani lime": "black lime",
    "omani limes": "black lime",
    "loomi": "black lime",

    "veg oil": "vegetable oil",
    "olive oils": "olive oil",

    "cilantro": "coriander",
    "coriander leaves": "coriander",
    "green coriander": "coriander",
    "parsley leaves": "parsley",
    "mint leaves": "mint",
    "spring onion": "green onion",
    "capsicum": "bell pepper",
    "green pepper": "bell pepper",
    "sweet pepper": "bell pepper",

    "turmeric powder": "turmeric",
    "ginger powder": "ginger",
    "garlic powder": "garlic",
    "onion powder": "onion",
    "cinnamon stick": "cinnamon",
    "cinnamon sticks": "cinnamon",
    "cardamon": "cardamom",

    "tomato puree": "tomato paste",
    "tomato purée": "tomato paste",
    "simple syrup": "sugar syrup",

    "garbanzo bean": "chickpeas",
    "garbanzo beans": "chickpeas",
    "chick pea": "chickpeas",
    "chick peas": "chickpeas",
    "black eyed bean": "black-eyed beans",
    "black eyed beans": "black-eyed beans",
}

# If BOTH appear, drop the generic in favor of the preferred.
# (You can add more pairs here.)
COLLAPSE_IF_PRESENT = [
    ("oil", "olive oil"),          # keep "olive oil", drop "oil"
    ("vegetable oil", "olive oil"),# keep "olive oil"
    ("pepper", "chili"),           # if chili is present, drop generic "pepper"
]

# Words to drop if they sneak in (not ingredients)
NON_ING_WORDS = {
    "after","decorate","decoration","decorations","add","with","such","touch","patriotic",
    "or","and","the","a","an","then","until","when","like","as","to","for","of","into","over",
    "warm","hot","cold","slice","sliced","diced","chopped","minced","ground","crushed","whole",
    "fresh","optional","needed","garnish","make","prepare","preparation","cook","cooked","baked",
    "boiled","fried","seauted","sauteed","browned","mix","mixed","topping","kitchen","precise","instant"
}

# ---------- helpers ----------
def parse_list_cell(x):
    """Parse list stored as Python-list string, or already-list, or fallback."""
    if isinstance(x, list):
        return [str(t).strip().lower() for t in x if str(t).strip()]
    s = str(x).strip()
    if not s:
        return []
    try:
        val = ast.literal_eval(s)
        if isinstance(val, list):
            return [str(t).strip().lower() for t in val if str(t).strip()]
    except Exception:
        pass
    # fallback: split by | or comma
    parts = re.split(r"\s*\|\s*|,+", s)
    return [p.strip().lower() for p in parts if p.strip()]

def apply_aliases(tokens):
    out = []
    for t in tokens:
        t0 = t.strip().lower()
        if not t0 or t0 in NON_ING_WORDS:
            continue
        t0 = ALIASES.get(t0, t0)
        out.append(t0)
    # de-dup preserve order
    seen, dedup = set(), []
    for x in out:
        if x not in seen:
            dedup.append(x); seen.add(x)
    return dedup

def collapse_generics(tokens):
    s = set(tokens)
    # if preferred present, drop generic
    for generic, preferred in COLLAPSE_IF_PRESENT:
        if preferred in s and generic in s:
            s.discard(generic)
    # rebuild original order
    out, seen = [], set()
    for x in tokens:
        if x in s and x not in seen:
            out.append(x); seen.add(x)
    return out

# ---------- load ----------
df = pd.read_csv(INPUT_CSV, encoding="utf-8")
missing = [c for c in REQ_COLS if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

# ---------- process ----------
changes = []
new_lists = []
for idx, row in df.iterrows():
    original = parse_list_cell(row[CLASS_COL])
    aliased  = apply_aliases(original)
    collapsed = collapse_generics(aliased)

    # track changes (only if different)
    if original != collapsed:
        changes.append({
            "row": idx,
            "before": json.dumps(original, ensure_ascii=False),
            "after":  json.dumps(collapsed, ensure_ascii=False),
        })
    new_lists.append(collapsed if collapsed else ["unknown_ingredients"])

df[CLASS_COL] = new_lists

# keep only requested 4 columns, same order
out = df[["dish_name", CLASS_COL, "image_file", "scrape_date"]]
out.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")

pd.DataFrame(changes).to_csv(REPORT_CSV, index=False, encoding="utf-8")

print("✅ Consistency pass complete.")
print(f"• Output CSV: {OUTPUT_CSV}")
print(f"• Changes report: {REPORT_CSV}")
print(f"Rows changed: {len(changes)}")





✅ Consistency pass complete.
• Output CSV: SaudiFoodFile_standardized_consistent.csv
• Changes report: consistency_changes_report.csv
Rows changed: 26
